In [30]:
try:
    %load_ext autoreload --quiet
except ImportError:
    %reload_ext autoreload 
%autoreload 2

from dataclasses import asdict
import yaml
from flow_merge.lib.merge_config import ValidatedInputData, MergeConfig
# from flow_merge.lib.tensor_loader import ModelSourceMetadata
from flow_merge.lib.model_metadata import load_model_info
from pprint import pprint
from huggingface_hub.hf_api import ModelInfo, RepoSibling, SafeTensorsInfo, BlobLfsInfo


file_path = "../examples/dare_ties_config.yaml"

with open(file_path, "r") as file:
    unvalidated_data = yaml.safe_load(file)

vd = ValidatedInputData(**unvalidated_data)
# mc = MergeConfig.from_yaml("../examples/dare_ties_config.yaml")
#md = ModelSourceMetadata.from_model_path("minghaowu/Qwen1.5-0.5B-OpenHermes-2.5", mc)

# print out nice outputs using export
print("\n")
pprint(vd)
print("\n")
# pprint(mc)
print("\n")
# pprint(md)

# rd = ModelSourceMetadata.fetch_from_hf("minghaowu/Qwen1.5-0.5B-OpenHermes-2.5")
rd = load_model_info("Qwen/Qwen1.5-0.5B")

pprint(rd)


for i in rd.siblings:
    print(f"\n\n{i.blob_id}\n{i.rfilename}\n{i.size}")
    print(i)
    if i.lfs:
        print(i.lfs.sha256)
        print(i.lfs.pointer_size)

# if rd.transformers_info:
#     print(rd.transformers_info)
    
if rd.safetensors:
    print(rd.safetensors.parameters)
    print(rd.safetensors.total)
# print(rd.sha)
# print(rd.created_at)
# print(rd.last_modified)
# print(rd.private)
# print(rd.gated)
# print(rd.disabled)
# print(rd.mask_token)
# print(rd.model_index)

### IMMEDIATE TODOS
# 1. create the modelsourcemetadata
# -> take in the file metadata
# -> do the file checks

# 2. include it in MergeConfig




### LATER TODOS
## 1. upload function to import HF_TOKEN as optional argument, and choose on priority - to Arek
## 2. use token in the merge_config








ValidatedInputData(method=<MergeMethodIdentifier.DARE_TIES_MERGING: 'dare-ties-merging'>, method_global_parameters=MethodGlobalParameters(scaling_coefficient=0.7, normalize=False, p=0.7, t=None, top_k=None), base_model='Qwen/Qwen1.5-0.5B', models=[RawModelDict(path_or_id='Qwen/Qwen1.5-0.5B', weight=0.5), RawModelDict(path_or_id='Qwen/Qwen1.5-0.5B-chat', weight=0.3), RawModelDict(path_or_id='minghaowu/Qwen1.5-0.5B-OpenHermes-2.5', weight=0.3)], tokenizer_settings=TokenizerSettings(mode='base', interpolation_method='linear'), directory_settings=DirectorySettings(cache_dir=None, local_dir=PosixPath('/Users/minaamshahid/projects/flowrite/flow-merge/notebooks/models'), output_dir=PosixPath('/Users/minaamshahid/projects/flowrite/flow-merge/notebooks/ties_merging_model')), hf_hub_settings=HfHubSettings(token=None, trust_remote_code=False), device=None)






flow_merge.lib.model_metadata - INFO - Model metadata created!
flow_merge.lib.model_metadata - INFO - Model metadata created!
flow_merge.lib.model_metadata - INFO - Model metadata created!
flow_merge.lib.model_metadata - INFO - Model metadata created!
flow_merge.lib.model_metadata - INFO - Model metadata created!
flow_merge.lib.model_metadata - INFO - Model metadata created!
flow_merge.lib.model_metadata - INFO - Model metadata created!
flow_merge.lib.model_metadata - INFO - Model metadata created!


ModelInfo(id='Qwen/Qwen1.5-0.5B',
          author='Qwen',
          sha='8f445e3628f3500ee69f24e1303c9f10f5342a39',
          created_at=datetime.datetime(2024, 1, 22, 16, 30, 10, tzinfo=datetime.timezone.utc),
          last_modified=datetime.datetime(2024, 4, 5, 10, 38, 41, tzinfo=datetime.timezone.utc),
          private=False,
          gated=False,
          disabled=False,
          downloads=55923,
          likes=116,
          library_name='transformers',
          tags=['transformers',
                'safetensors',
                'qwen2',
                'text-generation',
                'pretrained',
                'conversational',
                'en',
                'license:other',
                'autotrain_compatible',
                'endpoints_compatible',
                'text-generation-inference',
                'region:us'],
          pipeline_tag='text-generation',
          mask_token=None,
          card_data={'language': ['en'], 'license': 'other', 'li

In [42]:
from transformers import AutoConfig
from pathlib import Path

base_path = "../models/Qwen"
p = "/Qwen1.5-0.5B"


v = Path(base_path + p).resolve()

AutoConfig.from_pretrained(v).to_dict()

{'vocab_size': 151936,
 'max_position_embeddings': 32768,
 'hidden_size': 1024,
 'intermediate_size': 2816,
 'num_hidden_layers': 24,
 'num_attention_heads': 16,
 'use_sliding_window': False,
 'sliding_window': 32768,
 'max_window_layers': 21,
 'num_key_value_heads': 16,
 'hidden_act': 'silu',
 'initializer_range': 0.02,
 'rms_norm_eps': 1e-06,
 'use_cache': True,
 'rope_theta': 1000000.0,
 'attention_dropout': 0.0,
 'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'torch_dtype': 'bfloat16',
 'use_bfloat16': False,
 'tf_legacy_loss': False,
 'pruned_heads': {},
 'tie_word_embeddings': True,
 'chunk_size_feed_forward': 0,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'cross_attention_hidden_size': None,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_length': 0,
 'do_sample': False,
 'early_stopping': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'temperatu

In [22]:
import hashlib
import os
from pathlib import Path

def calculate_git_lfs_oid(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as file:
        # TODO: confirm git lfs default chunk size (right now ~65KB)
        for chunk in iter(lambda: file.read(64 * 1024), b""):
            sha256_hash.update(chunk)
    oid = sha256_hash.hexdigest()
    return oid

base_path = "../models/Qwen"
p = "/Qwen1.5-0.5B"


v = Path(base_path + p).resolve()
print(v.exists())

files = [f for f in v.glob("*")]

print(files)

print(calculate_git_lfs_oid(files[1]))
# f4b55f917af273d0dc98b67ec249f6445dd385f5
# 80d64fccf954f022eb43b7425eb45a133cbb92c76a5f24d4cea6dbe3d45548d9 <-- Ours

True
[PosixPath('/Users/minaamshahid/projects/flowrite/flow-merge/models/Qwen/Qwen1.5-0.5B/model.safetensors'), PosixPath('/Users/minaamshahid/projects/flowrite/flow-merge/models/Qwen/Qwen1.5-0.5B/tokenizer_config.json'), PosixPath('/Users/minaamshahid/projects/flowrite/flow-merge/models/Qwen/Qwen1.5-0.5B/config.json'), PosixPath('/Users/minaamshahid/projects/flowrite/flow-merge/models/Qwen/Qwen1.5-0.5B/tokenizer.json'), PosixPath('/Users/minaamshahid/projects/flowrite/flow-merge/models/Qwen/Qwen1.5-0.5B/vocab.json')]
80d64fccf954f022eb43b7425eb45a133cbb92c76a5f24d4cea6dbe3d45548d9


In [49]:
from flow_merge.lib.model_metadata import load_model_info

p = "Qwen/Qwen1.5-0.5B"

model_metadata = load_model_info(path_or_id=p)
model_metadata

flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge.lib.model_metadata - INFO - Model not found in Hugging face. Inferring from local model directory.
flow_merge

ModelMetadata(id='Qwen/Qwen1.5-0.5B', sha=None, config={'vocab_size': 151936, 'max_position_embeddings': 32768, 'hidden_size': 1024, 'intermediate_size': 2816, 'num_hidden_layers': 24, 'num_attention_heads': 16, 'use_sliding_window': False, 'sliding_window': 32768, 'max_window_layers': 21, 'num_key_value_heads': 16, 'hidden_act': 'silu', 'initializer_range': 0.02, 'rms_norm_eps': 1e-06, 'use_cache': True, 'rope_theta': 1000000.0, 'attention_dropout': 0.0, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': 'bfloat16', 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'chunk_size_feed_forward': 0, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, 'do_sample': False, 'early_stopping': False, 'num_beams': 1, 'num_beam_groups': 1, 'diversity_penalty'